# Traitement des distances - repère absolu

Dans ce notebook, on définit des fonctions permettant de faire différents traitements sur les distances des atomes des molécules par rapport à quatre points fictifs (atomes fictifs) formant un repère orthonormé. Ces points sont placés en (0, 0, 0), (1, 0, 0), (0, 1, 0) et (0, 0, 1)


### Génération d'un tableau de coordonnées fictives

In [87]:
import numpy as np

atomes_nb = 6
atomes_nb_incl_fictif = atomes_nb + 1


# Génération d'une matrice de positions des atomes d'une molécule. Pour chaque atome, 
# une ligne contient 3 coordonnées x, y, z, chacune dans l'intervalle [-1,1[. 
def gen_pos(nb_atomes):
    return (np.random.random_sample(atomes_nb*3) * 2 - 1).reshape(-1,3)


positions = gen_pos(atomes_nb)
positions

array([[-0.71020892, -0.82019586,  0.14520503],
       [ 0.18313614,  0.61939908,  0.80627034],
       [-0.83113651, -0.10509212, -0.60422013],
       [ 0.08420021, -0.65030167, -0.35658573],
       [ 0.22280307, -0.48873879, -0.39623437],
       [ 0.83452218,  0.16800897,  0.00552223]])

### Calcul de la matrice compressée de distances

Pour chaque atome de la molécule, la matrice compressée de distances contient la distance de l'atome avec les quatre points du repère absolu.

#### Calcul de la matrice de distances compressée (méthode vectorielle)

In [88]:
def matrice_distances_compr(positions):
    """ Renvoie la matrice de distances compressée des positions des atomes passées en paramètres
    La matrice de distances compressée est définie de la façon suivante : pour chaque atome, on calcule
    la distance avec chaque point du repère. Une ligne i de la matrice (n,4) correspond aux distances
    de l'atome i avec chacun des quatre points du repère"""
    
    nb_at = len(positions)
    
    # On renvoie un tableau vide si la molécule est vide (car sinon vstack lève une exception)
    if nb_at == 0:
        return []
    
    repere = np.array([[0, 0, 0], [1, 0, 0], [0, 1, 0], [0, 0, 1]])
    repere = np.vstack([repere]*nb_at)

    positions = np.tile(positions, 4).reshape(4*nb_at, 3)
    
    return np.sqrt(np.sum(np.power(positions-repere, 2), 1)).reshape(nb_at, 4)


### Ajout du bruit

On définit ici une fonction prenant une matrice de coordonnées en entrée et renvoyant une matrice des coordonnées bruitées selon une loi normale de centre 0 et de variance 0,000833. Il s'agit d'une variance qui semble raisonnable par rapport aux déplacement que l'on applique aux atomes lorsqu'on optimise une molécule. En effet, cela revient à déplacer l'atome à une distance de moins de 0.05 angströms dans environ 68% des cas, et à une distance de moins de 0.1 angströms dans 95% des cas.

In [89]:
def positions_bruitees(positions):
    """ Ajoute du bruit à un tableau de positions et renvoie un tuple (positions bruitées, bruit)"""
    bruit = np.random.normal(loc=0.0, scale=0.028867, size=positions.shape)
    return ((positions + bruit), bruit)


#### Analyse statistique des différences de distances introduites par le bruitage pour tester sa conformité

On ajoute du bruit sur chaque coordonnée à l'origine du repère sur dix millions d'exemples et on analyse la distributions de distances obtenue. On constate que l'on obtient bien des résultats conformes à nos attentes.

In [1]:
import pandas as pd

origine = np.array([0, 0, 0])
origine = np.repeat(origine, 10000000).reshape(-1, 3)
pos_bruit = positions_bruitees(origine)

def calc_dist_origine(point):
    return math.sqrt(pow(point[0], 2) + pow(point[1], 2) + pow(point[2], 2))

distances = []

for pos in pos_bruit:
    distances.append(calc_dist_origine(pos))
    
distances = pd.Series(distances)
distances.describe(percentiles=[0.68, 0.95])

NameError: name 'np' is not defined

### Reconstruction des molécules

L'objectif ici est de fournir une fonction permettant de reconstruire une molécule m à partir des distances de chaque atome a(i) de m aux quatre points du repère (atomes fictifs). Les atomes fictifs sont placés aux points (0, 0, 0), (1, 0, 0), (0, 1, 0), (0, 0, 1).

La méthode de reconstruction de l'atome i est la suivante : on calcule l'intersection des sphères de centres af(0), af(1), af(2) et de rayons d(a(i)af(0)), d(a(i)af(1)), d(a(i)af(2)). Cela nous donne deux points p0 et p1, dont l'un des deux est la position de l'atome i. On utilise la distance d(a(i)af(3)) pour choisir la bonne solution parmi p0 et p1.

#### Fonction de calcul de distances entre deux points (méthode vectorielle)

In [ ]:
import numpy as np

def calcul_distance(pt1, pt2):
    """ Renvoie la distance entre deux points représentés par leurs coordonnées (x, y, z) dans deux tableaux
    de forme (1, 3)"""
    return np.sqrt(np.sum(np.square(np.diff(np.array([pt1, pt2]), axis=0))))

#### Fonction de calcul de la position d'un point à partir de ses distances avec les points fixes du repère (atomes fictifs)

In [ ]:
import math
def calcul_position(distances_abs_at):
    """ Calcul de la position d'un atome à partir d'un tableau (1, 4) de ses distances avec les points
    du repère (atomes fictifs) 
    Le calcul de la position s'effectue en utilisant le cas particulier dans lequel on a fixé les centres
    des sphères de la méthode suivante : https://en.wikipedia.org/wiki/Trilateration
    La méthode de calcul est la suivante : on calcule l'intersection entre les trois sphères
    de centres af(0), af(1) et af(2) de rayons d(a(i)af(0)), d(a(i)af(1)), d(a(i)af(2)), puis on utilise
    la distance d(a(i)af(3)) pour choisir la bonne solution parmi les deux proposées par l'intersection.
    Les atomes fictifs (centres des sphères) sont placés aux positions suivantes : 
    (0, 0, 0), (1, 0, 0), (0, 1, 0) et (0, 0, 1)
    """
    
    position = np.empty(shape=(3,))
    
    # Calcul de la coordonnée x
    position[0] = (pow(distances_abs_at[0], 2) - pow(distances_abs_at[1], 2) + 1 )*0.5

    # Calcul de la coordonnée y
    position[1] = (pow(distances_abs_at[0], 2) - pow(distances_abs_at[2], 2) + 1 )*0.5
    
    ## Calcul de la coordonnée z qui possède une solution négative et une solution positive ## 
    
    # On calcule d'abord z^2 et on lui assigne la valeur zéro s'il est négatif (z^2 peut être très 
    # légèrement négatif à cause du manque de précision des distances)
    zpos = math.sqrt(max(pow(distances_abs_at[0], 2) - pow(position[0], 2) - pow(position[1], 2), 0))
    
    # On créé la liste contenant les deux valeurs possibles pour z
    z_sols = [zpos, -zpos]
    
    # On itère sur la liste des positions de z et on enregistre la solution dont la distance avec le quatrième
    # atome fictif est la plus proche de la distance attendue
    diff_dist_min = float("inf")
    for z_val in z_sols:
        curr_diff = abs(calcul_distance(np.array([position[0], position[1], z_val]), np.array([0, 0, 1]))
                        - distances_abs_at[3])
        if curr_diff < diff_dist_min:
            best_z_sol = z_val
            diff_dist_min = curr_diff
            
    position[2] = best_z_sol
    return position
    

#### Définition de la fonction de reconstruction d'une molécule (méthode vectorielle mais non parallélisée)

In [ ]:
def reconstruction_molecule(distances_abs):
    distances_abs = distances_abs.reshape(-1, 4)
    return np.apply_along_axis(calcul_position, 1, distances_abs)

#### Vérification de la reconstruction d'un grand nombre de molécules

On va maintenant appliquer une vérification de la reconstruction sur n molécules. Pour chaque molécule, on génère des positions aléatoirement, on calcule la matrice de distances compressée sur ces positions et on reconstruit les positions à partir de cette matrice de distances.
On calcule la différence entre les matrices position pré et post traitement, et on vérifie qu'aucune valeur n'est en dessous du seuil donné en paramètre.
De plus, on calcule la matrice de distances compressée sur les nouvelles positions et on la compare à la matrice de distances compressée pré-traitement par la même méthode.

In [105]:

def check_n_reconstructions(iterations, nb_atomes, epsilon, check_positions=True):
    """ Teste la reconstruction de iterations molécules contenant chacune nb_atomes atomes. La reconstruction
    de chaque molécule est validée si la différence entre les distances attendues et les distances calculées
    après la reconstructions est inférieure à l'epsilon donné.
    Si le paramètre check_positions est vrai, alors on reconstruit la molécule une fois de plus à chaque
    itération et on vérifie que les deux reconstructions ont produit les mêmes positions à epsilon près
    """
    
    succes = True
    
    erreurs_pos = 0
    erreurs_dist = 0
    
    imprecisions_glob_dist = []
    imprecisions_glob_pos = []
    
    for i in range(n):

        succes_courant = True
        
        positions = gen_pos(atomes_nb)  # On génère les positions

        distances = matrice_distances_compr(positions)  # On calcule la matrice des distances
        
        # On reconstruit les positions à partir des distances
        positions_post_traitement = reconstruction_molecule(distances)

        # On calcule les nouvelles distances
        nouvelles_distances = matrice_distances_compr(positions_post_traitement)

        if check_positions:
        
            # On reconstruit les nouvelles positions afin de vérifier qu'on obtient les mêmes
            positions_post_traitement2 = reconstruction_molecule(nouvelles_distances)
            
            # On enregistre l'échec de la validation si les positions de la deuxième reconstruction sont
            # différentes des positions de la première reconstruction
        
            diff_pos = abs(positions_post_traitement - positions_post_traitement2)
            succes_courant = (diff_pos < epsilon).all()
            imprecisions_glob_pos.append(diff_pos.flatten())

            if not succes_courant:
                erreurs_pos += 1
            
            succes = succes and succes_courant
            
            succes_courant = True

        # On enregistre l'échec de la validation si les distances pré/post reconstruction diffèrent
        diff_dist = abs(distances - nouvelles_distances)
        succes_courant = (diff_dist < epsilon).all()
        imprecisions_glob_dist.append(diff_dist.flatten())

        if not succes_courant and check_positions:
            erreurs_dist += 1
            
            
        succes = succes and succes_courant            
            
    
    print("Vérification des distances")
    print("Nombre de molécules ayant au moins une imprécision supérieure au seuil : "+str(erreurs_dist))
    print("Imprécision max : "+str(np.amax(imprecisions_glob_dist)))
    print("Imprécision moyenne "+str(np.mean(imprecisions_glob_dist)))
    print("Imprécision médiane : "+str(np.median(imprecisions_glob_dist)))

    if check_positions:
        print()
        print("Vérification des positions")
        print("Nombre de molécules ayant au moins une imprécision supérieure au seuil : "+str(erreurs_pos))
        print("Imprécision max : "+str(np.amax(imprecisions_glob_pos)))
        print("Imprécision moyenne "+str(np.mean(imprecisions_glob_pos)))
    
    return succes
    


In [106]:
n = 100000  # molécules générées
atomes_nb = 150  # nombre d'atomes par molécule

epsilon_diff = 1e-8  # tolérance epsilon

print(check_n_reconstructions(n, atomes_nb, epsilon_diff))

Vérification des distances
Nombre de molécules ayant au moins une imprécision supérieure au seuil : 0
Imprécision max : 1.4543732884675364e-09
Imprécision moyenne 4.011691799934991e-16
Imprécision médiane : 0.0

Vérification des positions
Nombre de molécules ayant au moins une imprécision supérieure au seuil : 0
Imprécision max : 1.5411295592154317e-09
Imprécision moyenne 2.934761583881062e-16
True
